<a href="https://colab.research.google.com/github/LeeSeungwon89/Kaggle_Dacon_Practice/blob/main/2.%20Bike_Sharing_Demand_basline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [2]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json

# 참가한 대회 리스트를 확인합니다.
!kaggle competitions list

In [4]:
!kaggle competitions download -c bike-sharing-demand

  0% 0.00/189k [00:00<?, ?B/s]
100% 189k/189k [00:00<00:00, 66.2MB/s]


In [5]:
!ls

bike-sharing-demand.zip  kaggle.json  sample_data


In [6]:
!unzip bike-sharing-demand.zip

Archive:  bike-sharing-demand.zip
  inflating: sampleSubmission.csv    
  inflating: test.csv                
  inflating: train.csv               


# **1. 피처 엔지니어링**

In [44]:
import numpy as np
import pandas as pd
import random

np.random.seed(2022)
random.seed(2022)

# 최대 행렬 수를 설정합니다.
pd.set_option('display.max_column', 50)
pd.set_option('display.max_rows', 50)

# 데이터를 읽습니다.
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sampleSubmission.csv')

## **1.1. 이상치 레코드 제거**

'weather' 피처에서 값이 4인 피처를 제거하겠습니다.

In [45]:
train = train[train['weather']!=4]

## **1.2. 훈련 및 테스트 세트 결합**

피처 엔지니어링을 수행하기 위해 훈련 세트와 테스트 세트를 결합하겠습니다.

In [46]:
all_data = pd.concat([train, test], ignore_index=True)
all_data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3.0,13.0,16.0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8.0,32.0,40.0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5.0,27.0,32.0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3.0,10.0,13.0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17373,2012-12-31 19:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,NaN,NaN
17374,2012-12-31 20:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,NaN,NaN
17375,2012-12-31 21:00:00,1,0,1,1,10.66,12.880,60,11.0014,NaN,NaN,NaN
17376,2012-12-31 22:00:00,1,0,1,1,10.66,13.635,56,8.9981,NaN,NaN,NaN


## **1.3. 피처 분할**

'datetime' 피처를 연, 월, 시, 요일 피처로 분할하겠습니다. 사용하기에 부적합한 날짜, 일, 분, 초 피처는 생성하지 않겠습니다.

In [47]:
from datetime import datetime
import calendar

# 연, 월, 시 피처를 생성합니다.
all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])
all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1])
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])

# 날짜를 추출하고 날짜에 해당하는 요일을 숫자로 치환합니다.
all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])
all_data['day_of_week'] = all_data['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday())

# 날짜 피처를 삭제합니다.
all_data.drop('date', axis=1, inplace=True)

In [48]:
all_data.head(1)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour,day_of_week
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3.0,13.0,16.0,2011,01,00,5


## **1.4. 불필요한 피처 제거**

불필요한 피처를 제거하겠습니다.

In [49]:
feature_list = ['datetime', 'casual', 'windspeed', 'registered']
all_data.drop(feature_list, axis=1, inplace=True)

In [50]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17378 entries, 0 to 17377
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   season       17378 non-null  int64  
 1   holiday      17378 non-null  int64  
 2   workingday   17378 non-null  int64  
 3   weather      17378 non-null  int64  
 4   temp         17378 non-null  float64
 5   atemp        17378 non-null  float64
 6   humidity     17378 non-null  int64  
 7   count        10885 non-null  float64
 8   year         17378 non-null  object 
 9   month        17378 non-null  object 
 10  hour         17378 non-null  object 
 11  day_of_week  17378 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 1.6+ MB


# **2. 모델 훈련**

## **2.1. 데이터 준비**

데이터를 준비하겠습니다.

In [60]:
train_num = len(train) # 훈련 세트 개수를 지정합니다.
X_train_df = all_data[:train_num].drop('count', axis=1) # 훈련 세트를 지정합니다.
X_test_df = all_data[train_num:].drop('count', axis=1) # 테스트 세트를 지정합니다.
y_train = train['count'] # 타깃값을 지정합니다.

타깃값에 로그 변환을 적용하겠습니다.

In [61]:
log_y_train = np.log(y_train)

## **2.2. RMSLE 측정 함수 선언**

이 문제의 측정 지표는 RMSLE입니다. RMSLE를 계산하는 함수를 선언하겠습니다.



---

업데이트 중

---

